### Importing necessary libraries and data

In [47]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as py

In [48]:
dictionary = pd.read_csv('dictionary.csv')
winter = pd.read_csv('winter.csv')
summer = pd.read_csv('summer.csv')

In [49]:
dictionary.head()

,Country,Code,Population,GDP per Capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN


In [50]:
winter.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1924,Chamonix,Biathlon,Biathlon,"BERTHET, G.",FRA,Men,Military Patrol,Bronze
1,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, C.",FRA,Men,Military Patrol,Bronze
2,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, Maurice",FRA,Men,Military Patrol,Bronze
3,1924,Chamonix,Biathlon,Biathlon,"VANDELLE, André",FRA,Men,Military Patrol,Bronze
4,1924,Chamonix,Biathlon,Biathlon,"AUFDENBLATTEN, Adolf",SUI,Men,Military Patrol,Gold


In [51]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [52]:
dictionary.isnull().sum()

Country            0
Code               0
Population         5
GDP per Capita    25
dtype: int64

In [53]:
winter.isnull().sum()

Year          0
City          0
Sport         0
Discipline    0
Athlete       0
Country       0
Gender        0
Event         0
Medal         0
dtype: int64

In [54]:
summer.isnull().sum()

Year          0
City          0
Sport         0
Discipline    0
Athlete       0
Country       4
Gender        0
Event         0
Medal         0
dtype: int64

### Who was in top 3 athletes in summer and winter?

In [55]:
winter_Medals = winter.groupby(['Athlete', 'Medal'])['Sport'].count().reset_index().sort_values(by='Sport',ascending=False)

In [56]:
summer_Medals = summer.groupby(['Athlete', 'Medal'])['Sport'].count().reset_index().sort_values(by='Sport',ascending=False)

In [57]:
winter_Medals.head(3)

,Athlete,Medal,Sport
724,"DAEHLIE, Björn",Gold,8
384,"BJOERNDALEN, Ole Einar",Gold,8
2029,"KIRVESNIEMI, Harri",Bronze,6


In [58]:
summer_Medals.head(3)

,Athlete,Medal,Sport
18383,"PHELPS, Michael",Gold,18
13465,"LEWIS, Carl",Gold,9
17060,"NURMI, Paavo",Gold,9


In [60]:
winter=winter.merge(dictionary,left_on='Country',right_on='Code',how='left')
winter.head()
# winter=winter[['Year','City','Sport','Discipline','Athlete','Country_x','Gender','Event','Medal','Country_y']]
# winter.columns=[['Year','City','Sport','Discipline','Athlete','Code','Gender','Event','Medal','Country']]

,Year,City,Sport,Discipline,Athlete,Country_x,Gender,Event,Medal,Country_y,Code,Population,GDP per Capita
0,1924,Chamonix,Biathlon,Biathlon,"BERTHET, G.",FRA,Men,Military Patrol,Bronze,France,FRA,66808385.0,36205.568102
1,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, C.",FRA,Men,Military Patrol,Bronze,France,FRA,66808385.0,36205.568102
2,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, Maurice",FRA,Men,Military Patrol,Bronze,France,FRA,66808385.0,36205.568102
3,1924,Chamonix,Biathlon,Biathlon,"VANDELLE, André",FRA,Men,Military Patrol,Bronze,France,FRA,66808385.0,36205.568102
4,1924,Chamonix,Biathlon,Biathlon,"AUFDENBLATTEN, Adolf",SUI,Men,Military Patrol,Gold,Switzerland,SUI,8286976.0,80945.079219


In [64]:
medals=winter.groupby(['Athlete','Medal'])['Sport'].count().reset_index().sort_values(by='Sport',ascending=False)
medals=medals.drop_duplicates(subset=['Medal'],keep='first')
medals.columns=[['Athlete','Medal','Count']]
medals

,Athlete,Medal,Count
724,"DAEHLIE, Björn",Gold,8
2029,"KIRVESNIEMI, Harri",Bronze,6
932,"EHRIG, Andrea",Silver,5


In [61]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


### What is medal distribution per country?

In [62]:
medals_map=summer.groupby(['Country'])['Medal'].count().reset_index()
data = [ dict(
        type = 'choropleth',
        autocolorscale = False,
        colorscale = 'Picnic',
        reversescale = True,
        showscale = True,
        locations = medals_map['Country'],
        z = medals_map['Medal'],
        locationmode = 'Country'        
            )
       ]

layout = dict(geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(230, 247, 255)',
        projection = dict(
        type = 'Miller'
        ),
            ),
        )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False)

In [63]:
medals_map.head(20)

,Country,Medal
0,AFG,2
1,AHO,1
2,ALG,15
3,ANZ,29
4,ARG,259
5,ARM,11
6,AUS,1189
7,AUT,146
8,AZE,26
9,BAH,27
